<a href="https://colab.research.google.com/github/DM871/dm871.github.io/blob/master/notebooks/ff501.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Python Implementation

We import the Gurobi Python Module and other Python libraries. With Gurobi 9.1.1, a pip installation of the product will automatically include a size-limited (2000 variables, 2000 linear constraints, and 200 quadratic constraints) license that should work in a Docker container, which is used by Google Colab. We use a restricted set of students since the full data set would require more than 2000 variables.

In [1]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB


In [3]:

class Data:
    def __init__(self, dirname):
        self.projects, self.flatten_t2p = self.read_projects(dirname)
        self.students, self.std_x_prj, self.prj_x_std, self.desirability, self.priority = self.read_students(dirname)

    def read_projects(self, dirname):
        """ reads projects """
        filename="https://raw.githubusercontent.com/DM871/dm871.github.io/main/assets/projects.txt"
        #filename = dirname+"/projects.txt"
        df = pd.read_csv(filename, sep=";",header=None,names=["nr","team","title","min","max","type"],keep_default_na=False)
        df = df.set_index(df["nr"].astype(str)+df["team"])
        projects = df.to_dict(orient='index')
        flatten_t2p={str(projects[p]["nr"]):[i for i in projects if projects[i]["nr"]==projects[p]["nr"] ] for p in projects}

        return projects, flatten_t2p


    def read_students(self, dirname):
        """ reads students file"""
        filename="https://raw.githubusercontent.com/DM871/dm871.github.io/main/assets/students_small.txt"
        #filename = dirname+"/students_small.txt"
        df = pd.read_csv(filename, sep=";",header=None,names=["id","group","type","priorities"],keep_default_na=False)
        DT = df.set_index(df["id"]).to_dict(orient='index')

        std_values={}
        for student in DT:
            ranks = DT[student]["priorities"].split(",")
            std_values[DT[student]["id"]] = {}
            for r in range(len(ranks)):
                for idp in self.flatten_t2p[ranks[r]]: #idps:
                    std_values[DT[student]["id"]][idp] = (r+1, 2**max(8-r-1,0))
        students=list(std_values.keys())
        std_x_prj = {p:[x for x in std_values if p in std_values[x]] for p in self.projects}
        prj_x_std = {s:list(std_values[s].keys()) for s in std_values}
        desirability = {(s,p):std_values[s][p][1] if p in std_values[s] else 0 for s in std_values for p in self.projects}
        priority = {(s,p):std_values[s][p][0] if p in std_values[s] else 0 for s in std_values for p in self.projects}

        return students, std_x_prj, prj_x_std, desirability, priority

data = Data("./data")


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1123)>

In [4]:
import sys;
import math;


class Solution:
    def __init__(self, solution):
        if not type(solution) is dict:
            sys.exit("Type of solution must be a dict of students with a list of project assigned")
        self.solution = solution

    def check(self, data):
        print("\n\n== Check Solution ==")
        print(( "Numb. of students: " + str(len(data.students))))
        print(( "Numb. of projects: " + str(len(data.projects))))

        std_x_prj = {p: [s for s in self.solution if self.solution[s]==p] for p in data.projects}
        tmp = [len(std_x_prj[x]) for x in std_x_prj]
        print(("Numb. of teams created: "+str(len([x for x in tmp if x>0]))))

        # not all students assigned:
        std_not_assigned = len(data.students) - len(self.solution)
        print(( "Numb. of unassigned students: " + str(std_not_assigned)))
        if std_not_assigned!=0:
            sys.exit("Solution INFEASIBLE")

        # students only get projects they ranked
        projects_not_ranked = 0
        for s in self.solution:
            if self.solution[s] not in data.std_x_prj:
                        projects_not_ranked += 1
        print("Numb. of students in projects not in their priority list: " + str(projects_not_ranked ))
        if projects_not_ranked!=0: sys.exit("Solution INFEASIBLE")

        # team creation and cardinality
        teams = {p:[x for x in self.solution if self.solution[x]==p] for p in data.projects}
        ub_viol=len([x for x in teams if len(teams[x]) > data.projects[x]["max"]])
        print("Numb. of overfull teams: " + str(ub_viol))
        #lb_viol=len([x for x in teams if len(teams[x])<data.projects[x])
        #print("Numb. of underfilled teams: " + str(lb_viol))

        # group members are assigned to the same teams
        # diff_groups = 0
        # for g in data.groups:
        #     prjs = len(set([self.solution[x] for x in data.groups[g]]))
        #     if prjs != 1:
        #         diff_groups += 1
        # print("Numb. of students in the same group assigned to different teams: " + str(diff_groups))

        # count classical utility
        tot_util=0
        for s in self.solution:
            tot_util += data.desirability[s,self.solution[s]]


        priority = {s: data.priority[s,self.solution[s]] for s in self.solution }
        worst_priority = max([data.priority[s,self.solution[s]] for s in self.solution])
        priority_scale = {r: len([s for s in priority if priority[s]==r]) for r in range(1,(worst_priority+1))}

        print("\nTotal welfare: " + str(tot_util))

        print("\n".join(["Stds with "+str(x)+". priority: "+str(priority_scale[x]) for x in priority_scale]))



In [5]:

def solve_problem(data):
    m = gp.Model("SPAP")
    m.setParam(gp.GRB.param.Method, 0)

    ######### BEGIN: Write here your model
    x={}
    for s in data.students:
        for p in data.prj_x_std[s]:
            x[s,p]=m.addVar(ub=1,lb=0,obj=-data.desirability[s,p])

    for s in data.students:
        m.addConstr(gp.quicksum(x[s,p] for p in data.prj_x_std[s])==1)
    for p in data.projects:
        m.addConstr(gp.quicksum(x[s,p] for s in data.std_x_prj[p])<=data.projects[p]["max"])

    # The objective is to minimize the costs
    m.modelSense=gp.GRB.MINIMIZE
    ######### END
    m.update()
    m.write("model.lp")
    m.optimize()
    m.write("model.sol")

    ######### BEGIN: retrive the solution and put it in the dictionary assign {student: project}
    ######### example assign = {'7cf0bbe434': '28', '62f5cf2694': '44b', '90e362d09c': '24a',...
    def getSolution():
        assign = {}
        for s in data.students:
            for p in data.prj_x_std[s]:
                if x[s,p].X>0.9:
                    assign[s]=p
        return assign
    ######### END
    return m.objVal, getSolution()


In [6]:

#data.printAll()
objVal,assignment = solve_problem(data)
sol = Solution(assignment)
sol.check(data)

Restricted license - for non-production use only - expires 2022-01-13
Changed value of parameter Method to 0
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 234 rows, 1890 columns and 3780 nonzeros
Model fingerprint: 0x28dc3335
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 1 rows and 4 columns
Presolve time: 0.02s
Presolved: 233 rows, 1886 columns, 3770 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -2.2016000e+04   3.200000e+01   3.170161e+08      0s
     504   -2.0112000e+04   0.000000e+00   0.000000e+00      0s

Solved in 504 iterations and 0.03 seconds
Optimal objective -2.011200000e+04


== Check Solution ==
Numb. of students: 172
Numb. of projects: 62
Numb. of teams c